# Imports

In [1]:
import os
import sys
import pandas as pd

# AI content (GitHub Copilot, 02/07/2024), verified and adapted by Nicolas Huber.
src_directory: str = os.path.join(os.getcwd(), "..")
sys.path.append(src_directory)

from helpers import data_analyzer as data_analyzer
from algorithms import angle_analyzer as angle_analyzer
from helpers import data_visualizer as data_visualizer
from helpers import optimize_thresholds as optimizer
import constants as constants

# Variables

In [2]:
CSV_FILE = "/Users/nicolas/Downloads/test_angle_analyzer.csv"

# Initialisation

In [3]:
Optimizer: optimizer.ThresholdOptimizer = optimizer.ThresholdOptimizer(
    CSV_FILE,
    constants.R_VALUE_WEIGHT,
    constants.P_VALUE_WEIGHT,
    constants.STD_ERROR_WEIGHT,
    constants.OPTIMIZATION_LIMIT,
    constants.OPTIMIZATION_STEPS,
)
AngleAnalyzer: angle_analyzer.AngleAnalyzer = angle_analyzer.AngleAnalyzer(
    CSV_FILE,
    constants.ANGLE_PAST_THRESHOLD,
    constants.ANGLE_FUTURE_THRESHOLD,
    constants.ANGLE_THRESHOLD,
    constants.LINEAR_REGRESSION_THRESHOLD,
)
DataAnalyzer: data_analyzer.DataAnalyzer = Optimizer.construct_data_analyzer()
Visualizer: data_visualizer.DataVisualizer = data_visualizer.DataVisualizer()

# Data Import

In [4]:
data: pd.DataFrame = AngleAnalyzer.read_csv_file()

# Optimization

In [5]:
optimization = Optimizer.optimize_thresholds(data, DataAnalyzer, AngleAnalyzer)

Iteration f = 10, p = 10 from 4:
--> Testing thresholds...
--> Results:
----> ANGLE_PAST_THRESHOLD: 10
----> ANGLE_FUTURE_THRESHOLD: 10
----> AVERAGE_R_VALUE: 0.5175449878654688
----> AVERAGE_P_VALUE: 6.701246656526688e-06
----> AVERAGE_STD_ERR: 0.03359707851462494
----> SCORE: 0.3071652744938218
--> Appending results...
--> Preparing next iteration...

Iteration f = 10, p = 15 from 4:
--> Testing thresholds...


/Users/nicolas/Dropbox/3_Bildung/2 KZO/Stichwortverzeichnis/0 Maturarbeit/6 Wettbewerbe/1_SJf/3_Coaching/2_Projekt/software/flight-analyzer/src/executor/../helpers/optimize_thresholds.py:185: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat(


--> Results:
----> ANGLE_PAST_THRESHOLD: 10
----> ANGLE_FUTURE_THRESHOLD: 15
----> AVERAGE_R_VALUE: 0.5417478691429849
----> AVERAGE_P_VALUE: 3.2838485170747637e-06
----> AVERAGE_STD_ERR: 0.029394758423173736
----> SCORE: 0.3221082604889185
--> Appending results...
--> Preparing next iteration...

Iteration f = 15, p = 10 from 4:
--> Testing thresholds...
--> Results:
----> ANGLE_PAST_THRESHOLD: 15
----> ANGLE_FUTURE_THRESHOLD: 10
----> AVERAGE_R_VALUE: 0.5499468867151431
----> AVERAGE_P_VALUE: 3.5637364500074367e-06
----> AVERAGE_STD_ERR: 0.030190935951329934
----> SCORE: 0.3269479693130179
--> Appending results...
--> Preparing next iteration...

Iteration f = 15, p = 15 from 4:
--> Testing thresholds...
--> Results:
----> ANGLE_PAST_THRESHOLD: 15
----> ANGLE_FUTURE_THRESHOLD: 15
----> AVERAGE_R_VALUE: 0.5640762918947841
----> AVERAGE_P_VALUE: 8.312257353463449e-08
----> AVERAGE_STD_ERR: 0.026097411671770824
----> SCORE: 0.3358360090329213
--> Appending results...
--> Preparing next 

In [6]:
optimization

,angle_past_threshold,angle_future_threshold,average_r_value,average_p_value,average_std_err,score
0,10,10,0.517545,6.701247e-06,0.033597,0.307165
1,10,15,0.541748,3.283849e-06,0.029395,0.322108
2,15,10,0.549947,3.563736e-06,0.030191,0.326948
3,15,15,0.564076,8.312257e-08,0.026097,0.335836


In [7]:
print("Optimierte Werte:")
print(f"--> {Optimizer.past_threshold_optimized}")
print(f"--> {Optimizer.future_threshold_optimized}")

Optimierte Werte:
--> 15
--> 15
